# BQL Examples

In [8]:
# Imports
from polars_bloomberg import BQuery

In [9]:
# Basic example with signle data point
with BQuery() as bq:
    df_lst = bq.bql("get(px_last) for(['IBM US Equity', 'OMX Index'])")
df_lst[0]

ID,px_last,DATE,CURRENCY
str,f64,date,str
"""IBM US Equity""",230.82,2024-12-14,"""USD"""
"""OMX Index""",2556.343,2024-12-14,"""SEK"""


In [10]:
# Average PE ratio per sector
query = """
let(#avg_pe=avg(group(pe_ratio(), gics_sector_name()));)
get(#avg_pe)
for(members('SPX Index'))
"""
with BQuery() as bq:
    df_lst = bq.bql(query)
df_lst[0].head()

ID,#avg_pe,REVISION_DATE,AS_OF_DATE,PERIOD_END_DATE,ORIG_IDS,GICS_SECTOR_NAME()
str,f64,date,date,date,str,str
"""Communication Services""",26.251735,2024-11-14,2024-12-14,2024-09-30,null,"""Communication Services"""
"""Consumer Discretionary""",26.453212,2024-12-11,2024-12-14,2024-11-23,null,"""Consumer Discretionary"""
"""Consumer Staples""",19.49465,2024-12-13,2024-12-14,2024-11-24,null,"""Consumer Staples"""
"""Energy""",18.427215,2024-11-12,2024-12-14,2024-09-30,null,"""Energy"""
"""Financials""",37.082801,2024-11-26,2024-12-14,2024-09-30,null,"""Financials"""


In [11]:
# Duration and ZSpread for search results
query = """
let(#dur=duration(duration_type=MODIFIED);
    #zsprd=spread(spread_type=Z);)
get(name(), #dur, #zsprd)
for(filter(screenresults(type=SRCH, screen_name='@COCO'),
           ticker in ['SEB', 'SHBASS']))
"""

with BQuery() as bq:
    df_lst = bq.bql(query)
df = df_lst[0].join(df_lst[1], on="ID").join(df_lst[2], on=["ID", "DATE"])
df

ID,name(),#dur,DATE,#zsprd
str,str,f64,date,f64
"""YU819930 Corp""","""SEB 6 ¾ PERP""",5.37363,2024-12-14,308.810572
"""ZQ349286 Corp""","""SEB 5 ⅛ PERP""",0.395636,2024-12-14,185.980438
"""ZO703315 Corp""","""SHBASS 4 ⅜ PERP""",1.956536,2024-12-14,213.358921
"""ZO703956 Corp""","""SHBASS 4 ¾ PERP""",4.946231,2024-12-14,255.85428
"""BW924993 Corp""","""SEB 6 ⅞ PERP""",2.231859,2024-12-14,211.55125
"""YV402592 Corp""","""SEB Float PERP""",0.212973,2024-12-14,232.71


In [12]:
# Average OAS spread for Nvidia bonds per maturity bucket
query = """
let(
    #bins = bins(maturity_years,
                 [3,9,18,30],
                 ['(1) 0-3','(2) 3-9','(3) 9-18','(4) 18-30','(5) 30+']);
    #average_spread = avg(group(spread(st=oas),#bins));
)
get(#average_spread)
for(filter(bonds('NVDA US Equity', issuedby = 'ENTITY'),
           maturity_years != NA))
"""

with BQuery() as bq:
    df_lst = bq.bql(query)
df_lst[0]

ID,#average_spread,DATE,ORIG_IDS,#BINS
str,f64,date,str,str
"""(1) 0-3""",31.195689,2024-12-14,"""QZ552396 Corp""","""(1) 0-3"""
"""(2) 3-9""",59.580383,2024-12-14,null,"""(2) 3-9"""
"""(3) 9-18""",110.614416,2024-12-14,"""BH393780 Corp""","""(3) 9-18"""
"""(4) 18-30""",135.160279,2024-12-14,"""BH393781 Corp""","""(4) 18-30"""
"""(5) 30+""",150.713405,2024-12-14,"""BH393782 Corp""","""(5) 30+"""


In [13]:
# stocks with 20d EMA above 200d EMA and RSI > 70
with BQuery() as bq:
    df_lst = bq.bql(
        """
        let(#ema20=emavg(period=20);
            #ema200=emavg(period=200);
            #rsi=rsi(close=px_last());)
        get(name(), #ema20, #ema200, #rsi)
        for(filter(members('OMX Index'),
                    and(#ema20 > #ema200, #rsi > 55)))
        with(fill=PREV)
        """
    )
df = (
    df_lst[0]
    .join(df_lst[1], on="ID")
    .join(df_lst[2], on=["ID", "DATE", "CURRENCY"])
    .join(df_lst[3], on=["ID", "DATE"])
)
df

ID,name(),#ema20,DATE,CURRENCY,#ema200,#rsi
str,str,f64,date,str,f64,f64
"""ERICB SS Equity""","""Telefonaktiebolaget LM Ericsso""",90.094984,2024-12-14,"""SEK""",74.917219,57.454412
"""SKFB SS Equity""","""SKF AB""",214.383743,2024-12-14,"""SEK""",205.174139,58.403269
"""SEBA SS Equity""","""Skandinaviska Enskilda Banken""",153.680261,2024-12-14,"""SEK""",150.720922,57.692703
"""ASSAB SS Equity""","""Assa Abloy AB""",338.829971,2024-12-14,"""SEK""",316.8212,55.467329
"""SWEDA SS Equity""","""Swedbank AB""",217.380431,2024-12-14,"""SEK""",213.776784,56.303481


In [14]:
# Axes for Swedish AT1 bonds with bids
query = """
let(#ax=axes();)
get(name, #ax)
for(filter(bondsuniv(ACTIVE),
    crncy()=='USD' and
    basel_iii_designation() == 'Additional Tier 1' and
    country_iso() == 'SE' and
    is_axed('Bid') == True))
"""

with BQuery() as bq:
    df_lst = bq.bql(query)
df = df_lst[0].join(df_lst[1], on="ID")
df.head()

IndexError: list index out of range

In [ ]:
# Get Bond Universe from Equity Ticker
query = """
let(#rank=normalized_payment_rank();
    #oas=spread(st=oas);
    #nxt_call=nxt_call_dt();
    )
get(name(), #rank, #nxt_call, #oas)
for(filter(bonds('GTN US Equity'), series() == '144A'))
"""

with BQuery() as bq:
    df_lst = bq.bql(query)
df = df_lst[0].join(df_lst[1], on="ID").join(df_lst[2], on="ID").join(df_lst[3], on="ID")
df.head()

ID,name(),#rank,#nxt_call,#oas,DATE
str,str,str,date,f64,date
"""YX231113 Corp""","""GTN 10 ½ 07/15/29""","""1st Lien Secured""",2026-07-15,612.527259,2024-12-11
"""BS116983 Corp""","""GTN 5 ⅜ 11/15/31""","""Sr Unsecured""",2026-11-15,1164.436869,2024-12-11
"""AV438089 Corp""","""GTN 7 05/15/27""","""Sr Unsecured""",2024-12-17,397.46361,2024-12-11
"""ZO860846 Corp""","""GTN 4 ¾ 10/15/30""","""Sr Unsecured""",2025-10-15,1202.244214,2024-12-11
"""LW375188 Corp""","""GTN 5 ⅞ 07/15/26""","""Sr Unsecured""",2025-01-06,184.717971,2024-12-11


In [ ]:
# Total Return of GTN Bonds
query = """
let(#rng = range(-3M, 0D);
    #rets = return_series(calc_interval=#rng,per=W);
    )
get(#rets)
for(filter(bonds('GTN US Equity'), series() == '144A'))
"""

with BQuery() as bq:
    df_lst = bq.bql(query)
df = df_lst[0].pivot(on="ID", index="DATE", values="#rets")
df.tail()

DATE,YX231113 Corp,BS116983 Corp,AV438089 Corp,ZO860846 Corp,LW375188 Corp
date,f64,f64,f64,f64,f64
2024-11-13,-0.01427,-0.03175,-0.01165,-0.030335,0.001243
2024-11-20,-0.002037,-0.018293,0.0044,-0.012467,-0.002927
2024-11-27,0.006718,0.044034,0.016076,0.038166,0.000209
2024-12-04,0.001222,0.016511,-0.000664,0.013262,0.001091
2024-12-11,-0.000379,-0.021545,0.004619,-0.026725,0.001306


In [ ]:
# Revenue per segment
query = """
    let(#segment=segment_name();
        #revenue=sales_Rev_turn(fpt=q, fpr=range(2023Q3, 2024Q3));
        )
    get(#segment, #revenue)
    for(segments('GTN US Equity',type=reported,hierarchy=PRODUCT, level=1))
"""
with BQuery() as bq:
    df_lst = bq.bql(query)
df = (
    df_lst[1]
    .join(df_lst[0].select("ID", "#segment"), on="ID")
    .pivot(on="#segment", index="PERIOD_END_DATE", values="#revenue")
)
df

PERIOD_END_DATE,Broadcasting,Production Companies,Other,Adjustment
date,f64,f64,f64,f64
2023-09-30,7.83e8,2e7,1.6e7,null
2023-12-31,8.13e8,3.2e7,1.9e7,null
2024-03-31,7.8e8,2.4e7,1.9e7,null
2024-06-30,8.08e8,1.8e7,0.0,null
2024-09-30,9.24e8,2.6e7,1.7e7,null
